In [1]:
import os,shutil,pickle,tqdm,sys,random,re,string,pause, datetime,glob
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# # The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="0" 
import keras
import sent2vec
import seq2seq
from seq2seq.models import AttentionSeq2Seq
from seq2seq.models import Seq2Seq

import tensorflow as tf
import tensorboard as tb

import numpy as np
import pandas as pd
from tqdm import tqdm
from random import shuffle
from math import log, floor

from keras.utils import multi_gpu_model

# from keras import backend as K
from keras.models import *
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.utils import *
from keras.layers.advanced_activations import *
from keras import *
from keras.engine.topology import *
from keras.optimizers import *

import gensim
from gensim.models.word2vec import *
from keras.preprocessing.text import *
from keras.preprocessing.sequence import *

from sklearn.model_selection import *
from sklearn.decomposition import *
from sklearn.cluster import *
from sklearn.metrics.pairwise import *

# from collections import Counter
from keras.utils.generic_utils import *
from keras import regularizers
import unicodedata as udata
from keras.applications import *
from keras.preprocessing.image import *

from keras import backend 
from imblearn.ensemble import *
from imblearn.combine import *
# from python.keras import backend 
# Embedding(10,20)
from keras_transformer.extras import ReusableEmbedding, TiedOutputEmbedding
from keras_transformer.position import TransformerCoordinateEmbedding
from keras_transformer.transformer import TransformerACT, TransformerBlock
from keras_transformer.bert import (
    BatchGeneratorForBERT, masked_perplexity,
    MaskedPenalizedSparseCategoricalCrossentropy)

import keras_metrics as km
from keras_trans_mask import RemoveMask, RestoreMask

from keras_multi_head import *
from keras_self_attention import SeqSelfAttention

Using TensorFlow backend.
/home/leoqaz12/anaconda3/lib/python3.6/site-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)
/home/leoqaz12/anaconda3/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [2]:
from models import transformer_bert_model
from bpe import BPEEncoder
from utils import *

In [3]:
#Phase1
root_dir = 'data/tree-rep-profiles_o2o/'
train_emb_api,train_emb , train_rep_ans = pickle.load(open(root_dir + 'pids_o2o.pkl','rb'))
valid_emb_api,valid_emb, valid_rep_ans = pickle.load(open(root_dir + 'pids_valid.pkl','rb'))
test_emb_api,test_emb ,test_rep_ans = pickle.load(open(root_dir + 'pids_test.pkl','rb'))
exp_api,exp_emb = pickle.load(open(root_dir + 'pids_exp.pkl','rb'))
# print('train of sent2vec vector:',train_emb.shape,train_emb_api.shape,train_fam_ans.shape,train_rep_ans.shape)
# print('valid of sent2vec vector:',valid_emb.shape,valid_emb_api.shape,valid_fam_ans.shape,valid_rep_ans.shape)
train_rep_ans = np.expand_dims(train_rep_ans,axis=-1)
valid_rep_ans = np.expand_dims(valid_rep_ans,axis=-1)
test_rep_ans = np.expand_dims(test_rep_ans,axis=-1)
print('test of sent2vec vector:',test_emb.shape,test_emb_api.shape,test_rep_ans.shape)
emb_matrix = pickle.load(open('data/api_emb_matrix.pkl','rb'))

test of sent2vec vector: (508, 226, 768) (508, 226) (508, 226, 1)


In [4]:
# phase2 補load:
train_emb_api,train_emb , train_rep_ans = pickle.load(open(root_dir + 'pids_train_only.pkl','rb'))
train_rep_ans = np.expand_dims(train_rep_ans,axis=-1)

In [71]:
def _shuffle(X, X2 ,X3):
#     X3 = np.take(train_fam_ans,[0],axis=-1) #只train第幾個familiy
    randomize = np.arange(len(X))
    np.random.shuffle(randomize)
#     print(X.shape, Y.shape)
    return (X[randomize], X2[randomize],X3[randomize])

In [72]:
train_emb, train_emb_api, train_rep_ans = _shuffle(train_emb, train_emb_api, train_rep_ans)
print('train of sent2vec vector:',train_emb.shape,train_emb_api.shape,train_rep_ans.shape)
print('valid of sent2vec vector:',valid_emb.shape,valid_emb_api.shape,valid_rep_ans.shape)

train of sent2vec vector: (4104, 226, 768) (4104, 226) (4104, 226, 1)
valid of sent2vec vector: (453, 226, 768) (453, 226) (453, 226, 1)


## Model Hyper-Parameters

In [5]:
emb_dim = valid_emb.shape[-1] #被除數
num_heads = 48#除數，要整除
max_length = test_emb_api.shape[1] # max sequence length
# fam_num = train_fam_ans.shape[1]
vocabulary_size = emb_matrix.shape[0]-1
transformer_depth = 1
transformer_dropout = 0.1
l2_reg_penalty = 1e-6#1e-4
dp_rate = 0.01

traina = True #改
batch_size = 128 #改

In [6]:
constrain = keras.constraints.MinMaxNorm(min_value=0.0, max_value=0.0, rate=1.0, axis=0)
init = keras.initializers.Ones()
coordinate_embedding_layer = TransformerCoordinateEmbedding(
        transformer_depth , name='coordinate_embedding') #positional encoding
act_layer = TransformerACT(
            name='adaptive_computation_time')

transformer_block = TransformerBlock(
            name='transformer', num_heads=num_heads,
            residual_dropout=transformer_dropout,
            attention_dropout=transformer_dropout,
            # Allow bi-directional attention
            use_masking=False)
add_segment_layer = Add(name='add_segment')
l2_regularizer = (regularizers.l2(l2_reg_penalty) if l2_reg_penalty else None)

## Model Archeitecture

In [7]:
sentemb1 = Input(shape=(max_length,emb_dim),name='sent_emb')
sentemb = Masking(mask_value=0)(sentemb1)
#shape=(max_length,emb_dim),,batch_shape=(batch_size,max_length,emb_dim)
sent_ids1 = Input(shape=(max_length,), dtype='int32', name='sent_ids') # 輸入的api funvtion name ID
sent_ids = Masking(mask_value=0)(sent_ids1)
#shape=(max_length,),batch_shape=(batch_size,max_length)
api_emb = Embedding(vocabulary_size+1, emb_dim,weights=[emb_matrix],input_length=max_length
                    ,trainable=True,name='api_emb')(sent_ids) #改

segment_embeddings = Add()([sentemb,api_emb])

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
att_rnn = GRU(int(emb_dim/4),return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)(segment_embeddings)
att_rnn = BatchNormalization(name='bn')(att_rnn)
self_att = SeqSelfAttention(kernel_initializer=keras.initializers.lecun_normal(), #sigmoid af
                            attention_activation='selu',name='self_attention')(att_rnn)
# self_att = MultiHead(SeqSelfAttention(attention_activation='sigmoid',name='self_attention'),layer_num=2,name='Multi_Head')(att_rnn)
# multi_head = Reshape((max_length,-1))(self_att)
# multi_head = Lambda(lambda x: keras.backend.concatenate(x,axis=-2),name='reshape')(self_att)
rep_prediction = (
        TimeDistributed(Dense(1, name='0_1_predict', activation='sigmoid',trainable=True),name='out_rep') # hard_sigmoid
    (self_att))

In [9]:
model = Model(inputs=[sent_ids1,sentemb1], outputs=[rep_prediction]) #out
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sent_ids (InputLayer)           (None, 226)          0                                            
__________________________________________________________________________________________________
sent_emb (InputLayer)           (None, 226, 768)     0                                            
__________________________________________________________________________________________________
masking_2 (Masking)             (None, 226)          0           sent_ids[0][0]                   
__________________________________________________________________________________________________
masking_1 (Masking)             (None, 226, 768)     0           sent_emb[0][0]                   
__________________________________________________________________________________________________
api_emb (E

In [78]:
model.load_weights('./model/o2o_stage_gru_selfatt/byterep_1stStage_0706_gruatt_sent2vec.h5')
# model.load_weights('./model/o2o_130_gru_selfatt/byterep_130fam_0701_gruatt_sent2vec.h5')#,by_name=True) #改

model loss opt

In [11]:
def full_multi_label_metric(y_true, y_pred):
    comp = K.equal(y_true, K.round(y_pred))
    return K.cast(K.all(comp, axis=-1), K.floatx())
def f1_metric(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def binary_focal_loss(gamma=2., alpha=.25):
    """
    Binary form of focal loss.
      FL(p_t) = -alpha * (1 - p_t)**gamma * log(p_t)
      where p = sigmoid(x), p_t = p or 1 - p depending on if the label is 1 or 0, respectively.
    References:
        https://arxiv.org/pdf/1708.02002.pdf
    Usage:
     model.compile(loss=[binary_focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"], optimizer=adam)
    """
    def binary_focal_loss_fixed(y_true, y_pred):
        """
        :param y_true: A tensor of the same shape as `y_pred`
        :param y_pred:  A tensor resulting from a sigmoid
        :return: Output tensor.
        """
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))

        epsilon = K.epsilon()
        # clip to prevent NaN's and Inf's
        pt_1 = K.clip(pt_1, epsilon, 1. - epsilon)
        pt_0 = K.clip(pt_0, epsilon, 1. - epsilon)

        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) \
               -K.sum((1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return binary_focal_loss_fixed

def f1(y_true, y_pred):
    y_pred = K.round(y_pred)
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

def f1_loss(y_true, y_pred):
    
    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)
from keras.metrics import top_k_categorical_accuracy
def custom_acc1(y_true, y_pred):
    return top_k_categorical_accuracy(y_true, y_pred,k=3)
from keras.metrics import binary_accuracy
def bin_acc(y_true, y_pred):
    return binary_accuracy(y_true, y_pred)

def precision(y_true, y_pred):
    return tf.keras.metrics.Precision(y_true,y_pred)[1]
def recall(y_true, y_pred):
    return tf.keras.metrics.Recall(y_true,y_pred)[1]
def Hamming_loss(y_true, y_pred):
    tmp = K.abs(y_true-y_pred)
    return K.mean(K.cast(K.greater(tmp,0.5),dtype=float))
def hn_multilabel_loss(y_true, y_pred):
    # Avoid divide by 0
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # Multi-task loss
    return K.mean(K.sum(- y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred), axis=1))
# from sklearn.metrics import f1_score
# def f1_sk(y_true,y_pred):
#     score = f1_score(y_true=y_true, y_pred=y_pred, average='weighted')
#     return score

# 訓練參數
los = [hn_multilabel_loss,binary_focal_loss(alpha=.25, gamma=2)] # 1st stage.  f1_loss
#SINGLE
# los = [binary_focal_loss(alpha=.25, gamma=2)] #改
# los = [f1_loss]
# los = [hn_multilabel_loss]
los = [losses.binary_crossentropy]
# MML
'''los = []
for i in range(fam_num):
    los.append(binary_focal_loss(alpha=.25, gamma=2))
los = [losses.binary_crossentropy] + los'''


metric = {'out_rep': bin_acc,'family': f1_metric} # 1st stage. km.f1_score()
#SINGLE
metric = [f1_metric,bin_acc]
# metric = [km.f1_score(),bin_acc,km.binary_f1_score()]
# metric = {'RasMMA': 'acc'}
metric = [bin_acc] #改
#MML
'''metrics = []
for i in range(fam_num+1):
    metrics.append('acc')
# metrics = {}
# metrics['RasMMA'] = 'acc'
# for i in range(fam_num):
#     metrics['fam'+str(i)]='acc'
metric = metrics'''


loss_weight = [1,1] #stage1 0.95,0.05  #1st stage # 2nd stage [0.01,0.99]
#SINGLE
loss_weight = [1]
#MML
'''loss_weight = []
for i in range(fam_num):
    loss_weight.append(0.95)
loss_weight = [0.05] + loss_weight'''

learning_rate = 5e-4#2e-4 # 2nd stage: 1e-4 @1st:2e-4 0.002
# batch_size = 128 #32 #128

num_epochs = 1000
patien = 50

model_save_path = './model/o2o_stage_gru_selfatt/byterep_2ndStage_0706_gruatt_sent2vec.h5' #改
tensorboard_log_path = './logs/'+ model_save_path.split('/')[-1].split('.')[0]

In [12]:
# import tensorflow as tf
optimizer = optimizers.Adam(
            lr=learning_rate, beta_1=0.9, beta_2=0.999, amsgrad=False) #clipnorm=1. , clipvalue=1.
optimizer = keras.optimizers.Nadam(lr=learning_rate, clipvalue=1.) #改
# tf.keras.optimizers.Nadam
lr_scheduler1 = callbacks.LearningRateScheduler(
        CosineLRSchedule(lr_high=0.001, lr_low=1e-8, #learning_rate #改 #2nd stage增加
                         initial_period=num_epochs),
        verbose=1)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=int(patien/3),
                                      min_lr=1e-8,mode='min') 

model.compile(
            optimizer,
            loss=los,
            metrics=metric ,loss_weights=loss_weight)#{'word_predictions': masked_perplexity})

In [13]:
# save best, early stopping, 2 models ens weight:(best=0.8,last=0.2)
history = History()
stop_nan = callbacks.TerminateOnNaN()
model_callbacks = [
        callbacks.ModelCheckpoint(
            model_save_path, #val_f1_metric,max。val_family_f1_metric
            monitor='val_loss',mode='min' ,save_best_only=True, verbose=1,save_weights_only=True), #改
            EarlyStopping(patience=patien,monitor='val_loss',verbose=1,mode='min'),
        lr_scheduler,history,stop_nan ,lr_scheduler1
    ]
model_callbacks.append(callbacks.TensorBoard(tensorboard_log_path))

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [82]:
def training_batch(batch_size, X_train1, X_train2 , Y_train1):#, Y_train2):
    '''
    X_train1 = sent_ids: shape為(N, max_seq_length)
    X_train2 = sentemb: shape為(N,max_seq_length, word_embedding_size)
    Y_train1 = class_prediction: shape為(N, max_seq_length, 1)
    Y_train2 = family_prediction(stage2): shape為(N, fam_num)
    '''
    idx = np.arange(len(X_train1))
    np.random.shuffle(idx)

    while True:
        for i in idx:
            train_X1 = X_train1[idx[i]:idx[i]+batch_size]
            train_X2 = X_train2[idx[i]:idx[i]+batch_size]
            train_Y1 = Y_train1[idx[i]:idx[i]+batch_size]
#             train_Y2 = Y_train2[idx[i]:idx[i]+batch_size]
#             yield(train_X2,train_Y2)
#             yield ([train_X1,train_X2],[train_Y1,train_Y2]) #ori
            yield ([train_X1,train_X2],[train_Y1]) #改
            if i == idx[-1]:
                idx = np.arange(len(X_train1))
                np.random.shuffle(idx)
                break
            
#     data_size = X_train.shape[0]
#     ep = data_size / batch_size

In [83]:

H = model.fit_generator( #train_rep_ans
    generator=training_batch(batch_size=batch_size,X_train1=train_emb_api,X_train2=train_emb ,
                                             Y_train1=train_rep_ans)#,Y_train2=train_fam_ans) #Y_train2
#                     generator=training_batch(batch_size=batch_size,X_train1=valid_emb_api,X_train2=valid_emb ,
#                                              Y_train1=train_rep_ans,Y_train2=train_fam_ans)
                        , steps_per_epoch=int(np.ceil(len(train_emb_api)/batch_size)) ,
                    epochs=num_epochs,callbacks=model_callbacks
                   ,validation_data= ([valid_emb_api,valid_emb], [valid_rep_ans]) #ori
#                    ,validation_data= (valid_emb, valid_fam_ans) 
#                    ,validation_data= ([valid_emb_api,valid_emb], [valid_rep_ans]) #ori #改
#                    ,validation_data= ([valid_emb_api,valid_emb], [valid_rep_ans]+valid_Y2) #validY2
                    ,max_queue_size=10 #,class_weight=fam_weights#[None,fam_weights] #改
                    ,workers=10,use_multiprocessing=True   
                   ,shuffle=True,verbose=1)
model.save(model_save_path+"_all.h5")
#1st:train 0_1_prediction=0.14XX

/home/leoqaz12/.local/lib/python3.6/site-packages/keras/engine/training_generator.py:47: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/1000

Epoch 00001: LearningRateScheduler setting learning rate to 0.001.
33/33 [==============================] - 23s 700ms/step - loss: 0.9465 - bin_acc: 0.7702 - val_loss: 0.4947 - val_bin_acc: 0.7749

Epoch 00001: val_loss improved from inf to 0.49472, saving model to ./model/o2o_stage_gru_selfatt/byterep_2ndStage_0706_gruatt_sent2vec.h5
Epoch 2/1000

Epoch 00002: LearningRateScheduler setting learning rate to 0.0009999975326256032.
33/33 [==============================] - 16s 471ms/step - loss: 0.3710 - bin_acc: 0.8308 - val_loss: 0.4526 - val_bin_acc: 0.8119

Epoch 00002: val_loss improved from 0.49472 to 0.45259, saving model to ./model/o2o_stage_gru_selfatt/byterep_2ndStage_0706_gruatt_sent2vec.h5
Epoch 3/1000

Epoch 00003: LearningRateScheduler setting learning rate to 0.0009999901305267642.
33/33 [==============================] - 15s 470ms/step - loss: 0.3440 - bin_acc: 0.8540 - val_loss: 0.4194 - val_bin_acc: 0.8192

Epoch 00003: val_loss improved from 0.45259 to 0.4

33/33 [==============================] - 15s 470ms/step - loss: 0.3209 - bin_acc: 0.8686 - val_loss: 0.4493 - val_bin_acc: 0.8219

Epoch 00027: val_loss did not improve from 0.32368
Epoch 28/1000

Epoch 00028: LearningRateScheduler setting learning rate to 0.000998202360798868.
33/33 [==============================] - 16s 477ms/step - loss: 0.2927 - bin_acc: 0.8793 - val_loss: 0.3701 - val_bin_acc: 0.8486

Epoch 00028: val_loss did not improve from 0.32368
Epoch 29/1000

Epoch 00029: LearningRateScheduler setting learning rate to 0.0009980668239035406.
33/33 [==============================] - 15s 467ms/step - loss: 0.2857 - bin_acc: 0.8797 - val_loss: 0.3864 - val_bin_acc: 0.8442

Epoch 00029: val_loss did not improve from 0.32368
Epoch 30/1000

Epoch 00030: LearningRateScheduler setting learning rate to 0.0009979263713390868.
33/33 [==============================] - 16s 476ms/step - loss: 0.2693 - bin_acc: 0.8895 - val_loss: 0.3685 - val_bin_acc: 0.8547

Epoch 00030: val_loss did not 

33/33 [==============================] - 16s 471ms/step - loss: 0.3030 - bin_acc: 0.8726 - val_loss: 0.3532 - val_bin_acc: 0.8389

Epoch 00056: val_loss did not improve from 0.32033
Epoch 57/1000

Epoch 00057: LearningRateScheduler setting learning rate to 0.00099228224444293.
33/33 [==============================] - 16s 470ms/step - loss: 0.3292 - bin_acc: 0.8664 - val_loss: 0.7100 - val_bin_acc: 0.6937

Epoch 00057: val_loss did not improve from 0.32033
Epoch 58/1000

Epoch 00058: LearningRateScheduler setting learning rate to 0.0009920048927774417.
33/33 [==============================] - 15s 469ms/step - loss: 0.2837 - bin_acc: 0.8859 - val_loss: 0.5449 - val_bin_acc: 0.7995

Epoch 00058: val_loss did not improve from 0.32033
Epoch 59/1000

Epoch 00059: LearningRateScheduler setting learning rate to 0.00099172268527164.
33/33 [==============================] - 16s 470ms/step - loss: 0.2825 - bin_acc: 0.8837 - val_loss: 0.4763 - val_bin_acc: 0.8032

Epoch 00059: val_loss did not imp


Epoch 00084: val_loss improved from 0.30731 to 0.30708, saving model to ./model/o2o_stage_gru_selfatt/byterep_2ndStage_0706_gruatt_sent2vec.h5
Epoch 85/1000

Epoch 00085: LearningRateScheduler setting learning rate to 0.0009826909925084429.
33/33 [==============================] - 15s 466ms/step - loss: 0.3012 - bin_acc: 0.8763 - val_loss: 0.3284 - val_bin_acc: 0.8683

Epoch 00085: val_loss did not improve from 0.30708
Epoch 86/1000

Epoch 00086: LearningRateScheduler setting learning rate to 0.0009822788864418067.
33/33 [==============================] - 16s 471ms/step - loss: 0.2598 - bin_acc: 0.8931 - val_loss: 0.3160 - val_bin_acc: 0.8655

Epoch 00086: val_loss did not improve from 0.30708
Epoch 87/1000

Epoch 00087: LearningRateScheduler setting learning rate to 0.0009818620205266134.
33/33 [==============================] - 15s 467ms/step - loss: 0.3020 - bin_acc: 0.8707 - val_loss: 0.3165 - val_bin_acc: 0.8716

Epoch 00087: val_loss did not improve from 0.30708
Epoch 88/1000

E

Epoch 00028: LearningRateScheduler setting learning rate to 0.000998202360798868.
79/79 [==============================] - 36s 450ms/step - loss: 0.4248 - bin_acc: 0.8226 - val_loss: 0.4296 - val_bin_acc: 0.8253

Epoch 00028: val_loss improved from 0.43866 to 0.42955, saving model to ./model/o2o_stage_gru_selfatt/byterep_1stStage_0706_gruatt_sent2vec.h5

Epoch 00003: LearningRateScheduler setting learning rate to 0.0014999851957407985.
90/90 [==============================] - 39s 431ms/step - loss: 0.6088 - bin_acc: 0.6647 - val_loss: 0.5624 - val_bin_acc: 0.7292

Epoch 00003: val_loss did not improve from 0.53809

Epoch 00002: LearningRateScheduler setting learning rate to 0.0014999962989260677.
56/56 [==============================] - 23s 419ms/step - loss: 0.5530 - bin_acc: 0.7259 - val_loss: 0.5224 - val_bin_acc: 0.7585


In [109]:
H.epoch

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139]

# Evaluate
* 挑選threshold

In [14]:
model.load_weights(model_save_path)
model.summary()
model_save_path

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sent_ids (InputLayer)           (None, 226)          0                                            
__________________________________________________________________________________________________
sent_emb (InputLayer)           (None, 226, 768)     0                                            
__________________________________________________________________________________________________
masking_2 (Masking)             (None, 226)          0           sent_ids[0][0]                   
__________________________________________________________________________________________________
masking_1 (Masking)             (None, 226, 768)     0           sent_emb[0][0]                   
__________________________________________________________________________________________________
api_emb (E

'./model/o2o_stage_gru_selfatt/byterep_2ndStage_0706_gruatt_sent2vec.h5'

In [17]:
from keras.models import model_from_json, model_from_yaml
from sklearn.metrics import f1_score,recall_score,precision_score,hamming_loss , roc_auc_score

# pickle.dump(file=open(tensorboard_log_path + '/'+'byterep_gruatt_H_2stage.pkl','wb'),obj=H.history) 
# json_string = model.to_json()
# yaml_string = model.to_yaml()
# pickle.dump(file=open(tensorboard_log_path + '/'+'byterep_gruatt_arch_2stage.pkl','wb'),obj=(json_string,yaml_string))

In [19]:
ans_t = model.predict([test_emb_api,test_emb])
ans_v = model.predict([valid_emb_api,valid_emb])
print(ans_t.shape)

(508, 226, 1)


In [20]:
y_true = test_rep_ans.squeeze(axis=-1)
ans_t = ans_t.squeeze(axis=-1)
area_score_t = roc_auc_score(y_true,ans_t,average='micro')
y_true = valid_rep_ans.squeeze(axis=-1)
ans_v = ans_v.squeeze(axis=-1)
area_score_v = roc_auc_score(y_true,ans_v,average='micro')
print('valid roc auc score:',area_score_v,'test:',area_score_t)

valid roc auc score: 0.8866524230103138 test: 0.8787838534467494


ori highest: valid roc auc score: 0.894314815832789 test: 0.8725151558369375

In [21]:
score_list = []
score_list_t = []
f1_list_v = []
pre_list_v = []
rec_list_v = []
f1_list_t = []
pre_list_t = []
rec_list_t = []
hloss_list_v = []
hloss_list_t = []
threashold_list = []
max_score = 0
for i in tqdm(range(1,100)): #system: 500 (0.002)
    thr = i/100 # system: 500 (0.002)
    final_ans_t = []
    for sample in ans_t:
        sample_ans = []
        for value in sample:
            if value < thr: #0.26 #0.33
                sample_ans.append(0)
            else:
                sample_ans.append(1)
        final_ans_t.append(sample_ans)
    final_ans_t = np.array(final_ans_t)
#     print(final_ans_t.shape , sum(final_ans_t[0]))

    final_ans_v = []
    for sample in ans_v:
        sample_ans = []
        for value in sample:
            if value < thr: #0.26 #0.33
                sample_ans.append(0)
            else:
                sample_ans.append(1)
        final_ans_v.append(sample_ans)
    final_ans_v = np.array(final_ans_v)
#     print(final_ans_v.shape , sum(final_ans_v[0]))

    y_true = test_rep_ans.squeeze(axis=-1)
    recall_t = recall_score(y_true=y_true, y_pred=final_ans_t, average='micro')
    precision_t = precision_score(y_true=y_true, y_pred=final_ans_t, average='micro')
    f1_t = f1_score(y_true=y_true, y_pred=final_ans_t, average='micro')
    h_loss_t = hamming_loss(y_true,final_ans_t)
#     print('Test: (recall/precision/f1/h_loss)',recall_t ,precision_t, f1_t , h_loss_t)

    y_true = valid_rep_ans.squeeze(axis=-1)
    recall_v = recall_score(y_true=y_true, y_pred=final_ans_v, average='micro')
    precision_v = precision_score(y_true=y_true, y_pred=final_ans_v, average='micro')
    f1_v = f1_score(y_true=y_true, y_pred=final_ans_v, average='micro')
    h_loss_v = hamming_loss(y_true,final_ans_v)
#     score = (f1_v+1.5*f1_t-h_loss_v-1.5*h_loss_t)/2.5
    score = f1_v - h_loss_v
    score_t = f1_t - h_loss_t
    if score>max_score:
        print('Best choice threashold now is:',thr,f1_v,f1_t,h_loss_v,h_loss_t)
        max_score = score
    score_list.append(score)
    score_list_t.append(score_t)
    threashold_list.append(thr)
    f1_list_v.append(f1_v)
    pre_list_v.append(precision_v)
    rec_list_v.append(recall_v)
    f1_list_t.append(f1_t)
    pre_list_t.append(precision_t)
    rec_list_t.append(recall_t)
    hloss_list_v.append(h_loss_v)
    hloss_list_t.append(h_loss_t)
#     print('Valid: (recall/precision/f1/h_loss)=>',recall_v ,precision_v, f1_v , h_loss_v)
print(max_score)

 49%|████▉     | 49/99 [00:15<00:15,  3.25it/s]

Best choice threashold now is: 0.49 0.8848084764097301 0.8734031336586322 0.04056535583816836 0.04497073374677723


 52%|█████▏    | 51/99 [00:16<00:14,  3.32it/s]

Best choice threashold now is: 0.51 0.8848087553123524 0.8734898340045183 0.04050674949696224 0.044874921608250296


100%|██████████| 99/99 [00:30<00:00,  3.44it/s]

0.8443020058153902


In [23]:
score_df = pd.DataFrame(data={'threshold':threashold_list,'Precision_valid':pre_list_v,'Recall_valid':rec_list_v
                              ,'F1_valid':f1_list_v,'Hloss_valid':hloss_list_v
                              ,'score_valid':score_list,'Precision_test':pre_list_t,'Recall_test':rec_list_t,
                              'F1_test':f1_list_t,'Hloss_test':hloss_list_t
                              ,'score_test':score_list_t})
score_df.to_excel('data/tree-rep-profiles_o2o/threxp_F1_rec_pre_Hloss.xlsx',index=False)
score_df

,threshold,Precision_valid,Recall_valid,F1_valid,Hloss_valid,score_valid,Precision_test,Recall_test,F1_test,Hloss_test,score_test
0,0.01,0.185124,0.999179,0.312373,0.785501,-0.473128,0.185536,0.999611,0.312981,0.785433,-0.472452
1,0.02,0.187375,0.998742,0.315549,0.773662,-0.458113,0.187685,0.998783,0.315991,0.773901,-0.457910
2,0.03,0.188940,0.998031,0.317730,0.765360,-0.447630,0.188931,0.995425,0.317584,0.765644,-0.448059
3,0.04,0.189952,0.996280,0.319070,0.759314,-0.440243,0.189944,0.993673,0.318924,0.759590,-0.440666
4,0.05,0.190384,0.996171,0.319673,0.757126,-0.437452,0.190359,0.993625,0.319507,0.757517,-0.438010
5,0.06,0.190589,0.995952,0.319951,0.755992,-0.436041,0.190569,0.993430,0.319793,0.756376,-0.436583
6,0.07,0.190794,0.995569,0.320220,0.754772,-0.434552,0.190799,0.993041,0.320096,0.755026,-0.434930
7,0.08,0.190894,0.995350,0.320349,0.754156,-0.433807,0.190866,0.992505,0.320162,0.754390,-0.434228
8,0.09,0.190963,0.994967,0.320426,0.753599,-0.433173,0.190898,0.991921,0.320177,0.753893,-0.433716
9,0.10,0.190928,0.993709,0.320312,0.753043,-0.432731,0.190901,0.990802,0.320123,0.753231,-0.433109


In [24]:
score_df[score_df['threshold']== 0.51]

,threshold,Precision_valid,Recall_valid,F1_valid,Hloss_valid,score_valid,Precision_test,Recall_test,F1_test,Hloss_test,score_test
50,0.51,0.898815,0.871232,0.884809,0.040507,0.844302,0.881542,0.865583,0.87349,0.044875,0.828615


In [25]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sent_ids (InputLayer)           (None, 226)          0                                            
__________________________________________________________________________________________________
sent_emb (InputLayer)           (None, 226, 768)     0                                            
__________________________________________________________________________________________________
masking_2 (Masking)             (None, 226)          0           sent_ids[0][0]                   
__________________________________________________________________________________________________
masking_1 (Masking)             (None, 226, 768)     0           sent_emb[0][0]                   
__________________________________________________________________________________________________
api_emb (E

Best choice threashold now is: 0.5 0.8819430841984498 0.8658494218075338 0.043309107333497576 0.04946201141853316

Best choice threashold now is: 0.52 0.8817946990116802 0.8657549261390528 0.04321672004598834 0.049306470502122675
1.655026434602622
***
Best choice threashold now is: 0.482 0.885380311340092 0.8738796101288141 0.04041883998515306 0.04485750121942722
2.088494634719019

# Deprecated

In [107]:
def chk_assetion(root_dir,ans_d,pad_num=0.49756694,max_pad=0.49756694,min_pad=0.49756693): #改treshold
    '''
    檢查profile順序是否與embedding順序一致
    Input: root_dir
    Output: length list
    '''
    dev_length = []
    dev_path_li = []
    fam_dir = next(os.walk(root_dir))[1]
    for fam in tqdm(fam_dir):
        tree_dir = next(os.walk(root_dir +fam))[1]
        for tree in tree_dir:
            in_directory = root_dir + fam +  '/' + tree + '/'
            hl_list = next(os.walk(in_directory))[2]
            hl_list = [os.path.join(in_directory, f) for f in hl_list]
            hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
#             dev_path_li.extend(hl_list)
            for hl_f in hl_list:
                with open(hl_f,encoding='ISO 8859-1') as f: #X2
                    lines = f.read()
                lines = re.sub(r'[^\x00-\x7F]+','', lines)
                lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
                lines = re.sub('=\\n','',lines)
                lines = lines.splitlines()
                dev_length.append(len(lines))
                dev_path_li.append(hl_f)
            
    for ans,length in zip(ans_d,dev_length):
        assert ans[:length][-1] != pad_num
        assert ans[:length+1][-1] < max_pad
        assert ans[:length+1][-1] > min_pad
    return dev_path_li

In [108]:
valid_path_li = chk_assetion('./data/tree-rep-profiles_o2o/DEV/',ans_v)
test_path_li = chk_assetion('./data/tree-rep-profiles_o2o/TEST/',ans_t)

100%|██████████| 45/45 [00:00<00:00, 223.95it/s]


In [109]:
(test_path_li)

['./data/tree-rep-profiles_o2o/TEST/18.domaiq_0.8/G395/44fe487808dd8fc0ff048d416be41a9414b66003d15cd6f0d75251f921c537e1_3416.profile',
 './data/tree-rep-profiles_o2o/TEST/18.domaiq_0.8/G395/88a899e31e4d89952f807b9ee2f69c3344e2d0137976b4988e5e95d2c7a95333_3236.profile',
 './data/tree-rep-profiles_o2o/TEST/18.domaiq_0.8/G394/413876356b48a780ee82c4452a511d81065e84fc69b748c0752eb3064781d5b8_3280.profile',
 './data/tree-rep-profiles_o2o/TEST/18.domaiq_0.8/G394/0c1ce1ef6a41b78c59aa4a3096c8777c70ba06cfe102046fc6b7a9617a72cad3_3320.profile',
 './data/tree-rep-profiles_o2o/TEST/18.domaiq_0.8/G397/01519c4765be0d99fce9a9bb4aa338793ab4742294a63f16540c05b34f74c175_2936.profile',
 './data/tree-rep-profiles_o2o/TEST/18.domaiq_0.8/G397/6f3460e84dbbe9d2d1065f58b57e5ac048fe34584fe2a17b5909919013301cef_3404.profile',
 './data/tree-rep-profiles_o2o/TEST/85.ipamor_0.8/G102/6e19cb71a89604d4b65c48fdc70d7fb2d39c7e835eeb60f860f535ecdd3af813_3368.profile',
 './data/tree-rep-profiles_o2o/TEST/85.ipamor_0.8/G102/

In [ ]:
def get_all_add_emb(dev_emb_api,dev_emb,model=model):
    intermediate_layer_model = Model(inputs=model.inputs,
                                 outputs=model.layers[5].output)
    intermediate_output = intermediate_layer_model.predict([valid_emb_api,valid_emb])
    print('Emb shape:',intermediate_output.shape)
    return intermediate_output

In [111]:

intermediate_layer_model = Model(inputs=model.inputs,
                                 outputs=model.layers[5].output)
intermediate_output = intermediate_layer_model.predict([valid_emb_api,valid_emb])
intermediate_output.shape

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
sent_ids (InputLayer)           (None, 216)          0                                            
__________________________________________________________________________________________________
sent_emb (InputLayer)           (None, 216, 768)     0                                            
__________________________________________________________________________________________________
masking_2 (Masking)             (None, 216)          0           sent_ids[0][0]                   
__________________________________________________________________________________________________
masking_1 (Masking)             (None, 216, 768)     0           sent_emb[0][0]                   
__________________________________________________________________________________________________
api_emb (E

(451, 216, 768)

In [114]:
intermediate_output[0][10]

array([ 6.2618005e-01, -2.1089106e+00, -3.0442843e+00,  5.3742480e-01,
        3.0402291e+00,  1.4082639e+00, -3.9007258e-01, -1.2419176e+00,
       -3.1372221e+00, -3.2730191e+00,  5.5726153e-01, -5.0289059e-01,
        1.0976368e+00, -1.1940066e+00, -2.0536981e+00,  2.4708271e+00,
        2.3215792e+00,  3.9515114e+00,  1.9344985e+00, -2.2881236e+00,
        2.2857480e+00, -2.0793555e+00,  1.4051611e+00,  3.4353054e+00,
       -4.0642748e+00, -1.7742591e+00, -9.2830986e-01, -3.3688855e+00,
       -1.1665313e+00, -1.2051904e+00, -2.6467364e+00,  1.9879489e+00,
        2.2966270e+00, -7.6591301e-01, -1.3570929e+00,  1.1703807e+00,
       -3.3427026e+00, -2.9934566e+00, -1.3248649e-01,  2.1944325e+00,
        2.6791778e-01,  4.2409101e-01, -9.6809852e-01, -1.5632174e+00,
       -4.7623760e-01,  5.6784534e+00,  2.5199026e-01, -6.0443032e-01,
        1.0090823e+00,  3.4446137e+00, -6.5510547e-01, -3.1890869e-03,
       -3.0715849e+00, -5.2878947e+00, -1.7584171e+00, -1.5329256e+00,
      

In [64]:
dev_length = []
root_dir = './data/tree-rep-profiles_o2o/DEV/'
fam_dir = next(os.walk(root_dir))[1]
for fam in tqdm(fam_dir):
    tree_dir = next(os.walk(root_dir +fam))[1]
    for tree in tree_dir:
        in_directory = root_dir + fam +  '/' + tree + '/'
        hl_list = next(os.walk(in_directory))[2]
        hl_list = [os.path.join(in_directory, f) for f in hl_list]
        hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
        for hl_f in hl_list:
            with open(hl_f,encoding='ISO 8859-1') as f: #X2
                lines = f.read()
            lines = re.sub(r'[^\x00-\x7F]+','', lines)
            lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
            lines = re.sub('=\\n','',lines)
            lines = lines.splitlines()
            dev_length.append(len(lines))
#             print(len(lines))
#             break
test_length = []
root_dir = './data/tree-rep-profiles_o2o/TEST/'
fam_dir = next(os.walk(root_dir))[1]
for fam in tqdm(fam_dir):
    tree_dir = next(os.walk(root_dir +fam))[1]
    for tree in tree_dir:
        in_directory = root_dir + fam +  '/' + tree + '/'
        hl_list = next(os.walk(in_directory))[2]
        hl_list = [os.path.join(in_directory, f) for f in hl_list]
        hl_list = list(filter(lambda f: f.endswith(".profile"), hl_list))
        for hl_f in hl_list:
            with open(hl_f,encoding='ISO 8859-1') as f: #X2
                lines = f.read()
            lines = re.sub(r'[^\x00-\x7F]+','', lines)
            lines = re.sub(r'[\x1e\x7f\x15\x10\x0c\x1c]+','', lines)
            lines = re.sub('=\\n','',lines)
            lines = lines.splitlines()
            test_length.append(len(lines))

100%|██████████| 45/45 [00:00<00:00, 235.73it/s]


In [98]:
for ans,length in zip(ans_v,dev_length):
    assert ans[:length][-1] != 0.49756694
    assert ans[:length+1][-1] < 0.49756694
    assert ans[:length+1][-1] > 0.49756693

In [96]:
ans[:length+1][-1] > 0.49756693

True

In [83]:
ans[:105][-1]

0.49756694

In [69]:
ans_t[-1][:200][-1]

0.49756694